In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler



In [2]:
df=pd.read_excel('/home/la-mus/AI/Data/dataset_tp_ml.xlsx')
df.head()

,Opérateur,Quartier,Type réseau,Download (Mbps),Upload (Mbps),Latence (ms),Jitter (ms),Loss (%),Qualite
0,Airtel,CENTRE-VILLE,4G,46.1,17.2,49.0,2.0,1.0,Bonne
1,Airtel,CENTRE-VILLE,4G,7.8,6.9,83.0,7.0,1.0,Mauvaise
2,Airtel,Marché,3G,39.2,14.2,16.0,12.0,3.0,Bonne
3,VODACOM,Résidentiel,4G,48.2,7.6,12.0,3.0,0.0,Bonne
4,VODACOM,Résidentiel,4G,48.4,6.7,42.0,7.0,4.0,Bonne


ici on feras du pretraitement de notre modele

In [3]:
#verifions de absent de valeurs null
print("Valeurs nulles par colonne :\n")
df.isnull().sum()


Valeurs nulles par colonne :



Opérateur          0
Quartier           0
Type réseau        0
Download (Mbps)    0
Upload (Mbps)      1
Latence (ms)       0
Jitter (ms)        0
Loss (%)           0
Qualite            0
dtype: int64

In [4]:
# Supprimer valeurs manquantes
#s'il y a de valeurs manquate supriime de ligne vide 
df=df.dropna()
#ici on  vas transforme de colonne non numerique de ces tableu en nombre 
#pour que la machine puisses comprendre on vas l'applique sur type de reseau et texte x
#notre encodage est base sur quartier ,operateur ettype de reseau
le_Operateur =LabelEncoder()
le_Quartier=LabelEncoder()
le_Type=LabelEncoder()

df["Opérateur"]=le_Operateur.fit_transform(df["Opérateur"])
df["Quartier"]=le_Operateur.fit_transform(df["Quartier"])
df["Type réseau"]=le_Operateur.fit_transform(df["Type réseau"])
#on vas en code la variable cible car la machine ne comprends pas du text
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(df["Qualite"])
#ici notre resultat en attente ces type de reseau cible 
#bonne=0 moyenne=1 mauvaise=2

ici on serparera x et y 

In [5]:
#toute le colonne serons separe  sauf la cible
X=df.drop("Qualite", axis =1)
#colonne cible
y=df["Qualite"]

In [6]:
#encoder la cible
le = LabelEncoder()
y = le.fit_transform(y)
print(y[:5]) 

[0 1 0 0 0]


In [7]:
#ici on vas faire la normalisation de toute les colonnes sauf la colonne cible
scaler = MinMaxScaler()
colonnes_num = ["Download (Mbps)", "Upload (Mbps)", "Latence (ms)", "Jitter (ms)", "Loss (%)"]
X[colonnes_num] = scaler.fit_transform(X[colonnes_num])
#verifions 
X.head()

,Opérateur,Quartier,Type réseau,Download (Mbps),Upload (Mbps),Latence (ms),Jitter (ms),Loss (%)
0,2,6,20,2.269395e-12,8.449809e-13,0.014724,0.001873,0.136986
1,2,6,20,3.790551e-13,3.366104e-13,0.024940,0.006554,0.136986
2,2,90,9,1.928838e-12,6.969119e-13,0.004808,0.011236,0.410959
3,5,106,20,2.373043e-12,3.711599e-13,0.003606,0.002809,0.000000
4,5,106,20,2.382914e-12,3.267391e-13,0.012620,0.006554,0.547945


In [8]:
# ici on vas faire le sauvegarde d'un modle vide qui n'as pas appris de donne 
#car nous avons pas fait l'entrainment donc on feraas pas la prediction ici
# Création du modèle (configuration définie)
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
# Sauvegarde du modèle vide
joblib.dump(clf, "modele_non_entraine.pkl")


['modele_non_entraine.pkl']

ici on vas entraine le modole avec le NN  dons le couche

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(
    #ici x et y sont nos variable
X,y,
test_size=0.2,
random_state=42)

In [10]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

y_train = y_train.astype("int32")
y_test = y_test.astype("int32")


In [11]:
clf=RandomForestClassifier(n_estimators=100,random_state=42)
#entrainement du model
clf.fit(X_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


CREONS NOTRE RESEAU 

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2026-02-19 23:25:55.334055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
#ici on defis notre reseau ces couche 
#ces focntion d'activation

model= keras.Sequential(
[
    layers.Dense(128,activation="relu",input_shape=(X_train.shape[1],)),
    layers.Dense(32,activation='relu'),
    layers.Dense(3,activation='softmax')
])

/home/la-mus/AI/lib/python3.13/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
#complite le modele
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [41]:
#ici on fait l'entrainement du  reeseau 

history=model.fit(
    X_train,
    y_train,
    #ici le modele feras 200 passege sur chaque donne
    epochs=45,
    #ici on vas divise le doonne en lot de 32lignes
    batch_size=32,
    # on feras en sorte que l'accuracy et loss fasses leurs donnes de test
    validation_data=(X_test,y_test)
    
)

Epoch 1/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7286 - loss: 0.6499 - val_accuracy: 0.5930 - val_loss: 0.8181
Epoch 2/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6669 - loss: 0.6718 - val_accuracy: 0.6105 - val_loss: 0.7905
Epoch 3/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7040 - loss: 0.6075 - val_accuracy: 0.5756 - val_loss: 0.8761
Epoch 4/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6769 - loss: 0.7172 - val_accuracy: 0.5756 - val_loss: 0.8895
Epoch 5/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6820 - loss: 0.6793 - val_accuracy: 0.6105 - val_loss: 0.8167
Epoch 6/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7628 - loss: 0.6063 - val_accuracy: 0.5756 - val_loss: 0.8139
Epoch 7/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7176 - loss: 0.6379 - val_accuracy: 0.5930 - val_loss: 0.8145
Epoch 8/45
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6723 - loss: 0.7037 - val_accuracy: 0.6105 - val_los

In [42]:
# Évaluer le modèle sur le jeu de test
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy sur le jeu de test :", accuracy)
print("Loss sur le test :", loss)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5876 - loss: 0.8418 
Accuracy sur le jeu de test : 0.6162790656089783
Loss sur le test : 0.8377255201339722


l'interpretation de mes resutlat le modele aprends ou predit correctement de 65% de cas sur les donnes de test et il ces trompe de 40% des cas est si on fait le 1/3=0.33 de hasard donc notre accurary est meilleurs que les hasard qu'on peu trouver , c'est la veux dire que le modele atteint une accuracy de 65% il clase correctement de 65% d'observation cette prediction est superieur au hasard le modle peut etre ameliore pour augmente la capacite de generalisation

NOUS AVONS DE RAISONE POUR QUE NOTRE MODEL COMPRENDS PAS BIEN EST :
1 LE DONNE sont peut 
2 sont bruite 
3 classes desequilibre
4 les hyperparametre non auptimise 

le train accuracy ici on veut dire que le modele  classe correctement environs 68% des donnees d'entrainement 

In [43]:
# scaler = MinMaxScaler() que tu as utilisé
joblib.dump(scaler, "scaler.pkl")
print("Scaler sauvegardé dans scaler.pkl")

Scaler sauvegardé dans scaler.pkl
